# ライブラリをクロージャで書いてみる

Pythonだとクラスより軽いと聞いたので...

## 累積和
ライブラリにするほどでもないんだけど...

In [19]:
# クラスで記述

class Sum_class:
    """
    size      : int
        元配列の大きさ
    arr       : [int]
        元配列
    s         : [int]
        累積和配列
    """

    def __init__(self, arr):
        self.size = len(arr)
        self.arr = arr
        self.s = [0] * (self.size + 1)
        for i in range(self.size):
            self.s[i+1] = self.s[i] + self.arr[i]
    
    def sum(self, x, y):
        """
        x <= i < y を満たすiに対して、sum a_i を求める。
        """
        return self.s[y] - self.s[x]

In [20]:
# クロージャで記述

def Sum_closure(arr):
    n = len(arr)
    s = [0] * (n+1)
    for i in range(n):
        s[i+1] = s[i] + arr[i]
    
    def sum_(x, y):
        return s[y] - s[x]
    
    return sum_

### テスト

In [14]:
import random
random.seed(0)

def get_random_range(N):
    x, y = random.randint(0, N), random.randint(0, N)
    if x > y:
        x, y = y, x
    return (x, y)

In [15]:
N = 100000
Q = 100000

arr = [random.randint(0, 100000) for _ in range(N)]
queries = [get_random_range(N) for _ in range(Q)]

In [17]:
%%timeit

# 普通の累積和
s1 = [0] * (N + 1)
for i in range(N):
    s1[i+1] = s1[i] + arr[i]

for x, y in queries:
    s1[y] - s1[x]

15.5 ms ± 1.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
%%timeit

# class版累積和
s2 = Sum_class(arr)

for x, y in queries:
    s2.sum(x, y)

24.1 ms ± 883 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
%%timeit

# クロージャ
s3 = Sum_closure(arr)

for x, y in queries:
    s3(x, y)

17.7 ms ± 906 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## クロージャの中身を変更

BITがどれだけ早くなるか試してみる

In [31]:
class BIT:
    def __init__(self, N):
        self.size = N
        self.arr = [0] * (N+1)

    def add(self, i, x):
        while i <= self.size:
            self.arr[i] += x
            i += i & -i

    def sum(self, i):
        res = 0
        while i:
            res += self.arr[i]
            i -= i & -i
        return res

In [36]:
def BIT_closure(size):
    N = size
    arr = [0] * (N+1)

    def add(i, x):
        while i <= N:
            arr[i] += x
            i += i & -i
    
    def sum(i):
        res = 0
        while i > 0:
            res += arr[i]
            i -= i & -i
        return res
    
    return (add, sum)

### テスト ###
# https://judge.u-aizu.ac.jp/onlinejudge/description.jsp?id=DSL_2_B&lang=jp
if __name__ == "__main__":
    N, Q = map(int, input().split())
    bit = BIT_closure(N)

    for i in range(Q):
        com, x, y = map(int, input().split())
        if com == 0:
            bit[0](x, y)
        elif com == 1:
            print(bit[1](y) - bit[1](x-1))

0


In [ ]:
from collections import namedtuple

def BIT_closure(size):
    N = size
    arr = [0] * (N+1)

    def add(i, x):
        while i <= N:
            arr[i] += x
            i += i & -i
    
    def sum(i):
        res = 0
        while i > 0:
            res += arr[i]
            i -= i & -i
        return res
    
    BIT_ = namedtuple("BIT", "add sum")
    bit = BIT_(add, sum)
    return bit

### テスト ###
# https://judge.u-aizu.ac.jp/onlinejudge/description.jsp?id=DSL_2_B&lang=jp
if __name__ == "__main__":
    N, Q = map(int, input().split())
    bit = BIT_closure(N)

    for i in range(Q):
        com, x, y = map(int, input().split())
        if com == 0:
            bit.add(x, y)
        elif com == 1:
            print(bit.sum(y) - bit.sum(x-1))

うーん。微妙
今後は普通にクラス使うかなー